# Credit Card Data Analysis for Customer Approval

### Ask Phase



1. What type of company does your client represent, and what are they asking you to accomplish?
    - The client represent a bank, based on the credit card data bank needs to make decision on which customers would benifit from the credit card and which customers would be unable to pay back the money. based on the previous customer data bank need to make decision on upcoming customers to approve credit cards in such a way that it won't be a burden to the company.

2. What are the key factors involved in the business task you are investigating?
    - For the current scenario client has provided a dataset of credit card application data, which contains card approved or not, age, number of derogatory reports, ownership of house, etc. by analysing this dataset we need pull out insights about the card holders.
        Below are the key factors that under investigation:
        - Who are the customers maintaining credit card account without failing the repayment
        - Who are the customers failing to repay the amount
    Based on these two factors more sub queries has to be made, asking underlying reason for the outcome would make a clear picture.

3. What type of data will be appropriate for your analysis?
    - client has provided credit card application dataset which would be used for this analysis.

4. Where will you obtain that data?
    - Client provided data

5. Who is your audience, and what materials will help you present to them effectively?
    - We will be working with financial analyst, other stakeholder like financial department manager and CFO will be involved in this project


### Prepare Phase



1. Where is your data located?
    - Google BigQuery
2. How is the data organized?
    - As a table (only one data set is available)
3. Are there issues with bias or credibility in this data? Does your data ROCCC?
    - Yes it does satisfies the ROCCC concepts
4. How are you addressing licensing, privacy, security, and accessibility?
    - Data given by client so all of them are considered by the client
5. How did you verify the data’s integrity?
    - Provided by the client
6. How does it help you answer your question?
    - Dataset contains all required columns so it would be sufficient to answer the questions
7. Are there any problems with the data?
    - Data needs to be cleaned since it contains null values and format of the data

### Process Phase


1. For the processing of data duplicated the table with a name process_data
> Create table `creditcard_dataset.processed_data` as (select * from `creditcard_dataset.creditcard`)



2. The age column is in decimal format which is not usually used, it is changed to integer by flooring the age
> UPDATE `creditcard_dataset.processed_data`
SET age = FLOOR(age)
WHERE TRUE



3. Few rows contains age = 0 which is not logically correct, so deleting the rows since it doesn't significantly impact the analysis
> DELETE FROM `creditcard_dataset.processed_data`
where age < 18



4. The income column is devided by 10,000, we don't need that right now so converting it back to the normal form.
> Update `creditcard_dataset.processed_data`
set income = income * 10000
where TRUE



5. changing the decimal points to 2 since we don't need so many trailing zeros after the decimal points
> Update `creditcard_dataset.processed_data`
set income = ROUND(income, 2)
where TRUE



6. share column seems to be incorrect where it is the ratio of monthly expenditure devided by anual income
> Update `creditcard_dataset.processed_data`
set share = expenditure/income
where TRUE



7. There are many accounts with more than 10 active credit accounts which is suspecious and not practically attainable from a financial perspective, we need to check summarised count of active accounts
> select active, count(active)
from `creditcard_dataset.processed_data`
group by active

8. Checking the number of major cards with each customer
> select majorcards, count(majorcards)
from `creditcard_dataset.processed_data`
group by majorcards

### Since all the data is cleaned for analysis we can proceed to the analysis phace, **but we might have to exclude active column due to inconsitency in the data**.

### Analysis Phase